In [ ]:
import boto3, botocore
from botocore.exceptions import ClientError
import os, time, json, io, zipfile
from datetime import date
from dotenv import load_dotenv


from misc import load_from_yaml, save_to_yaml
import iam, s3, lf, rds, vpc, ec2

load_dotenv(".env")
# boto3.setup_default_session(profile_name="AMominNJ")

In [ ]:
ALL_IN_ONE_SG = 'sg-0d8a868137f653df6'
ACCOUNT_ID        = os.environ['AWS_ACCOUNT_ID_ROOT']
REGION            = os.environ['AWS_DEFAULT_REGION']
VPC_ID            = os.environ['AWS_DEFAULT_VPC']
SECURITY_GROUP_ID = os.environ['AWS_DEFAULT_SG_ID']
SUBNET_IDS        = SUBNET_IDS = os.environ["AWS_DEFAULT_SUBNET_IDS"].split(":")
SUBNET_ID         = SUBNET_IDS[0]
AWS_INSTANCE_ID_JMASTER   = os.environ['AWS_INSTANCE_ID_JMASTER']
AWS_INSTANCE_ID_JAGENT    = os.environ['AWS_INSTANCE_ID_JAGENT']
AWS_DEFAULT_IMAGE_ID      = os.environ['AWS_DEFAULT_IMAGE_ID']
AWS_DEFAULT_KEY_PAIR_NAME = os.environ['AWS_DEFAULT_KEY_PAIR_NAME']
AWS_DEFAULT_INSTANCE_TYPE = os.environ['AWS_DEFAULT_INSTANCE_TYPE']

In [ ]:
sts_client           = boto3.client('sts')
rds_client           = boto3.client('rds')
iam_client           = boto3.client('iam')
s3_client            = boto3.client('s3')
glue_client          = boto3.client('glue')
lakeformation_client = boto3.client('lakeformation')
stepfunctions_client = boto3.client('stepfunctions')
apigateway_client    = boto3.client('apigateway')
lsn_client           = boto3.client('lambda')
events_client        = boto3.client('events')
sqs_client           = boto3.client('sqs')

emr_client = boto3.client('emr', region_name=REGION)

In [ ]:
ec2_client           = boto3.client('ec2', region_name=REGION)
ec2_resource         = boto3.resource('ec2', region_name=REGION)

# Example: Get a specific VPC
vpc = ec2_resource.Vpc('vpc_id')

# Example: Get a specific EBS volume
volume = ec2_resource.Volume('volume_id')

# [EC2](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/ec2.html)

## Fundamentals

In [ ]:
# !scp /Users/am/mydocs/Software_Development/noteshub/dotfiles/lnx/automate_installations.sh jenkins_agent1:/home/ubuntu/

In [ ]:
# Launch EC2 instance with tagging using TagSpecifications
response = ec2_client.run_instances(
    ImageId='ami-04b70fa74e45c3917',  # Replace with your AMI ID
    InstanceType='t2.micro', #'t2.medium', 't2.micro'
    MinCount=1,
    MaxCount=1,
    KeyName='AMominNJ',               # Replace with your key pair
    TagSpecifications=[
        {
            'ResourceType': 'instance',
            'Tags': [{'Key': 'Name', 'Value': 'JENKINS_AGENT'}]
        }
    ],
    BlockDeviceMappings=[
        {
            'DeviceName': '/dev/sda1',  # Default root volume
            'Ebs': {
                'VolumeSize': 10,  # Volume size in GiB
                'VolumeType': 'gp2'        # General Purpose SSD
            }
        }
    ],
    SecurityGroupIds=[ALL_IN_ONE_SG],
    # SubnetId=subnet_id if subnet_id else None
)

# Extract the instance ID of the newly created instance
instance_id = response['Instances'][0]['InstanceId']
print(f"Instance created with ID: {instance_id}")

In [ ]:
response = ec2_client.describe_instances(InstanceIds=['i-032ac00bc29763d73'])
print(response['Reservations'][0]['Instances'][0]['PublicDnsName'])
print(response['Reservations'][0]['Instances'][0]['PublicIpAddress'])

In [ ]:
# # Stop the instance immediately after creation
ec2_client.stop_instances(InstanceIds=[os.environ['AWS_INSTANCE_ID_JMASTER']])

In [ ]:
ec2_client.start_instances(InstanceIds=[AWS_INSTANCE_ID_JAGENT])

In [ ]:
ec2_client.terminate_instances(InstanceIds=[AWS_INSTANCE_ID_JAGENT])

- **Elastic IP**

In [ ]:
# Step 2: Allocate an Elastic IP
print("Allocating an Elastic IP...")
allocation = ec2_client.allocate_address(Domain='vpc')
elastic_ip = allocation['PublicIp']
allocation_id = allocation['AllocationId']
print(f"Elastic IP allocated: {allocation_id}")
print(f"Elastic IP allocated: {elastic_ip}")


In [ ]:
addresses = ec2_client.describe_addresses()
for address in addresses['Addresses']:
    print(f"Public IP: {address['PublicIp']}, Allocation ID: {address.get('AllocationId', 'N/A')}")

In [ ]:
ec2_client.release_address(AllocationId="eipalloc-0397507c5325a58c0")

In [ ]:

# Step 3: Associate the Elastic IP with the instance
# print(f"Associating Elastic IP {elastic_ip} with instance {instance_id}...")
ec2_client.associate_address(InstanceId="i-0a593d44df00aa8f9",AllocationId="eipalloc-03cd6f9a89e292ffb")
# print(f"Elastic IP {elastic_ip} associated with instance {instance_id}.")


In [ ]:
ec2_client.disassociate_address(AssociationId='eipassoc-0abcdef1234567890')


- **Key Pair (Identity File)**

In [ ]:
response = ec2_client.create_key_pair(
    KeyName='string',
    KeyType='rsa'|'ed25519',
    TagSpecifications=[
        {
            'ResourceType': 'capacity-reservation'|'client-vpn-endpoint'|'customer-gateway'|'carrier-gateway'|'coip-pool'|'dedicated-host'|'dhcp-options'|'egress-only-internet-gateway'|'elastic-ip'|'elastic-gpu'|'export-image-task'|'export-instance-task'|'fleet'|'fpga-image'|'host-reservation'|'image'|'import-image-task'|'import-snapshot-task'|'instance'|'instance-event-window'|'internet-gateway'|'ipam'|'ipam-pool'|'ipam-scope'|'ipv4pool-ec2'|'ipv6pool-ec2'|'key-pair'|'launch-template'|'local-gateway'|'local-gateway-route-table'|'local-gateway-virtual-interface'|'local-gateway-virtual-interface-group'|'local-gateway-route-table-vpc-association'|'local-gateway-route-table-virtual-interface-group-association'|'natgateway'|'network-acl'|'network-interface'|'network-insights-analysis'|'network-insights-path'|'network-insights-access-scope'|'network-insights-access-scope-analysis'|'placement-group'|'prefix-list'|'replace-root-volume-task'|'reserved-instances'|'route-table'|'security-group'|'security-group-rule'|'snapshot'|'spot-fleet-request'|'spot-instances-request'|'subnet'|'subnet-cidr-reservation'|'traffic-mirror-filter'|'traffic-mirror-session'|'traffic-mirror-target'|'transit-gateway'|'transit-gateway-attachment'|'transit-gateway-connect-peer'|'transit-gateway-multicast-domain'|'transit-gateway-policy-table'|'transit-gateway-route-table'|'transit-gateway-route-table-announcement'|'volume'|'vpc'|'vpc-endpoint'|'vpc-endpoint-connection'|'vpc-endpoint-service'|'vpc-endpoint-service-permission'|'vpc-peering-connection'|'vpn-connection'|'vpn-gateway'|'vpc-flow-log'|'capacity-reservation-fleet'|'traffic-mirror-filter-rule'|'vpc-endpoint-connection-device-type'|'verified-access-instance'|'verified-access-group'|'verified-access-endpoint'|'verified-access-policy'|'verified-access-trust-provider'|'vpn-connection-device-type'|'vpc-block-public-access-exclusion'|'ipam-resource-discovery'|'ipam-resource-discovery-association'|'instance-connect-endpoint'|'ipam-external-resource-verification-token',
            'Tags': [
                {
                    'Key': 'string',
                    'Value': 'string'
                },
            ]
        },
    ],
    KeyFormat='pem'|'ppk',
    DryRun=True|False
)

In [ ]:
# Get the list of key pairs
response = ec2_client.describe_key_pairs()

# Print key pair names and details
for key_pair in response['KeyPairs']:
    print(f"Key Name: {key_pair['KeyName']}, Key Fingerprint: {key_pair['KeyFingerprint']}")

- Instance Profile

In [ ]:
response = iam_client.list_instance_profiles()

# Extract and return the instance profiles
instance_profiles = response["InstanceProfiles"]

In [ ]:
profile_name = "MyInstanceProfile"  # Name for the instance profile
role_name = "MyEC2Role"            # Name of the existing IAM role
instance_id = "i-0abcdef1234567890" # Replace with your EC2 instance ID

In [ ]:
# Create the instance profile
print(f"Creating instance profile: {profile_name}")
iam_client.create_instance_profile(InstanceProfileName=profile_name)

# Attach the role to the instance profile
print(f"Attaching role '{role_name}' to instance profile '{profile_name}'")
iam_client.add_role_to_instance_profile(
    InstanceProfileName=profile_name,
    RoleName=role_name
)

In [ ]:
ec2_client.associate_iam_instance_profile(
    IamInstanceProfile={
        'Name': profile_name
    },
    InstanceId=instance_id
)

In [ ]:
def create_instance_profile(profile_name, role_name, policy_arn):
    iam_client = boto3.client('iam')

    try:
        # Step 1: Create IAM Role
        print(f"Creating IAM Role: {role_name}")
        assume_role_policy_document = {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Principal": {"Service": "ec2.amazonaws.com"},
                    "Action": "sts:AssumeRole"
                }
            ]
        }

        iam_client.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json.dumps(assume_role_policy_document)
        )

        print(f"IAM Role '{role_name}' created successfully.")

        # Step 2: Attach Policy to the Role
        print(f"Attaching policy {policy_arn} to role {role_name}.")
        iam_client.attach_role_policy(
            RoleName=role_name,
            PolicyArn=policy_arn
        )

        print(f"Policy '{policy_arn}' attached to role '{role_name}' successfully.")

        # Step 3: Create an Instance Profile
        print(f"Creating Instance Profile: {profile_name}")
        iam_client.create_instance_profile(InstanceProfileName=profile_name)
        print(f"Instance Profile '{profile_name}' created successfully.")

        # Step 4: Add Role to Instance Profile
        print(f"Adding role '{role_name}' to instance profile '{profile_name}'.")
        iam_client.add_role_to_instance_profile(
            InstanceProfileName=profile_name,
            RoleName=role_name
        )
        print(f"Role '{role_name}' added to instance profile '{profile_name}' successfully.")

    except iam_client.exceptions.EntityAlreadyExistsException as e:
        print(f"Entity already exists: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    import json

    # Specify the instance profile name, role name, and policy ARN
    instance_profile_name = "MyEC2InstanceProfile"
    role_name = "MyEC2Role"
    policy_arn = "arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"  # Example policy for S3 read-only access

    create_instance_profile(instance_profile_name, role_name, policy_arn)


## Security Group

In [ ]:
group_name = "ALL_IN_ONE_SG"

In [ ]:
# Example usage
inbound_rules = [
    {
        'IpProtocol': 'tcp',
        'FromPort': 22,
        'ToPort': 22,
        'IpRanges': [{'CidrIp': '0.0.0.0/0', "Description": "SSH_Port"}]
    },
    {
        'IpProtocol': 'tcp',
        'FromPort': 80,
        'ToPort': 80,
        'IpRanges': [{'CidrIp': '0.0.0.0/0', "Description": "HTTP_Port"}]
    },
    {
        'IpProtocol': 'tcp',
        'FromPort': 443,
        'ToPort': 443,
        'IpRanges': [{'CidrIp': '0.0.0.0/0', "Description": "HTTPs_Port"}]
    },
    {
        'IpProtocol': 'tcp',
        'FromPort': 8000,
        'ToPort': 8000,
        'IpRanges': [{'CidrIp': '0.0.0.0/0', "Description": "Django_Port1"}]
    },
    {
        'IpProtocol': 'tcp',
        'FromPort': 8010,
        'ToPort': 8010,
        'IpRanges': [{'CidrIp': '0.0.0.0/0', "Description": "Django_Port2"}]
    },
    {
        'IpProtocol': 'tcp',
        'FromPort': 8080,
        'ToPort': 8080,
        'IpRanges': [{'CidrIp': '0.0.0.0/0', "Description": "Jenkins_Port"}]
    },
    {
        'IpProtocol': 'tcp',
        'FromPort': 8888,
        'ToPort': 8888,
        'IpRanges': [{'CidrIp': '0.0.0.0/0', "Description": "JupyterNotebook_Port"}]
    },
    {
        'IpProtocol': 'tcp',
        'FromPort': 3306,
        'ToPort': 3306,
        'IpRanges': [{'CidrIp': '0.0.0.0/0', "Description": "AWS_RDS_MySQL_Port"}]
    },
    {
        'IpProtocol': 'tcp',
        'FromPort': 4243,
        'ToPort': 4243,
        'IpRanges': [{'CidrIp': '0.0.0.0/0', "Description": "docker_host"}]
    },
    {
        'IpProtocol': 'tcp',
        'FromPort': 32768,
        'ToPort': 60999,
        'IpRanges': [{'CidrIp': '0.0.0.0/0', "Description": "docker_host_communication"}]
    },
]

outbound_rules = [
    {
        'IpProtocol': '-1', # '-1' means all protocols
        'FromPort': -1,
        'ToPort': -1,
        'IpRanges': [{'CidrIp': '0.0.0.0/0'}]
    }
]

In [ ]:
security_group_id = ec2_client.create_security_group(
    GroupName=group_name,
    Description="ALL IN ONE SECURITY GROUP",
    VpcId=VPC_ID
)['GroupId']

In [ ]:
ec2_client.authorize_security_group_ingress(
    GroupId=security_group_id,
    IpPermissions=inbound_rules
)

# Adds an inbound rule to allow all traffic from instances within the same security group.
ec2_client.authorize_security_group_ingress(
    GroupId=security_group_id,
    IpPermissions=[
        {
            'IpProtocol': '-1',  # '-1' means all protocols
            'UserIdGroupPairs': [
                {
                    'GroupId': security_group_id,
                    'Description': 'Allow all traffic from the same security group'
                }
            ]
        }
    ]
)

In [ ]:
ec2_client.authorize_security_group_egress(GroupId=security_group_id,IpPermissions=outbound_rules)

In [ ]:
ec2_client.describe_security_groups(GroupIds=[security_group_id])

In [ ]:
tags = [{'Key': 'Name','Value': 'ALL_IN_ONE_SG'}]
ec2_client.create_tags(Resources=[security_group_id],Tags=tags)

In [ ]:

# Retrieve the current security groups attached to the instance
response = ec2_client.describe_instances(InstanceIds=[AWS_INSTANCE_ID_JENKINS_MASTER])
current_security_groups = response['Reservations'][0]['Instances'][0]['SecurityGroups']

# Extract current security group IDs
security_group_ids = [sg['GroupId'] for sg in current_security_groups]
print(security_group_ids)

- Update Security group of a EC2 instance.

In [ ]:
# Update the instance to attach the new list of security groups
ec2_client.modify_instance_attribute(
    InstanceId="i-064a3505ab360d46d",
    Groups=[ALL_IN_ONE_SG]
)

## VPC

-   [AWS re:Invent 2020: Monitoring and troubleshooting network traffic](https://www.youtube.com/watch?v=Ed09ReWRQXc)

#### [Must Know: AWS Networking Basics For Programmers | Hands On](https://www.youtube.com/watch?v=2doSoMN2xvI&t=1007s)

- VPC can have different types of Gateways
  - Internet Gateway
  - NAT Gateway: 
    - It is usally created in a public subnet
    - Only allow outbound trafic (Usually used to update packages by the instances within a private subnet)
  - Interface Gateway

- Each VPC have a Routers.
- Each VPC have a Default Route Table (Called `Main`).
  - It allows all internal communication `WITHIN` (subnet to subnet) the VPC
  - All subnets without any explecit association get associated to the `Main` route table.


- Each Subnet has a Route Table


- Route Table (RT):
  - By Default RT allwo communication amoung the subnets within the VPC.
  - RTs are created within a VPC (TR belongs to VPC)
  - Main Route Table: all unassociated subnets use the route table.
  - RT gets associated with Subnets.

<div style="text-align:center" ><img src="./screenshots/vpc/screenshot.png" width="700" height="400" /></div>

In [ ]:
# ?ec2_resource.create_vpc

In [ ]:
vpc_cidr_block = '172.0.0.0/16'
vpc_name = 'httx-vpc'
vpc_id = ec2_client.create_vpc(CidrBlock=vpc_cidr_block)['Vpc']['VpcId']

# Add a Name tag to the VPC
ec2_client.create_tags(Resources=[vpc_id], Tags=[{'Key': 'Name', 'Value': vpc_name}])
# ec2_client.describe_vpcs(VpcIds=[vpc_id])

In [ ]:
vpc_endpoints = ec2_client.describe_vpc_endpoints(Filters=[{'Name': 'vpc-id', 'Values': [vpc_id]}])
print(vpc_endpoints)
# for endpoint in vpc_endpoints['VpcEndpoints']:
#     endpoint_id = endpoint['VpcEndpointId']
#     print(f"Deleting VPC Endpoint: {endpoint_id}")
#     ec2_client.delete_vpc_endpoints(VpcEndpointIds=[endpoint_id])


In [ ]:

vpc = ec2_resource.Vpc(vpc_id)

In [ ]:
# ?ec2_resource.create_subnet

In [ ]:
subnet_configs = [
    {'cidr_block': '172.0.1.0/24', 'az': 'us-east-1a', 'tag': 'public-subnet-01'},
    {'cidr_block': '172.0.2.0/24', 'az': 'us-east-1b', 'tag': 'public-subnet-02'},
    {'cidr_block': '172.0.3.0/24', 'az': 'us-east-1a', 'tag': 'private-subnet-01'},
    {'cidr_block': '172.0.4.0/24', 'az': 'us-east-1b', 'tag': 'private-subnet-02'}
]

In [ ]:
public_subnet1 = ec2_resource.create_subnet(
    CidrBlock=subnet_configs[0]['cidr_block'],
    VpcId=vpc.id,
    AvailabilityZone=subnet_configs[0]['az']
)
ec2_client.create_tags(Resources=[public_subnet1.id],Tags=[{'Key': 'Name', 'Value': subnet_configs[0]['tag']}])

public_subnet2 = ec2_resource.create_subnet(
    CidrBlock=subnet_configs[1]['cidr_block'],
    VpcId=vpc.id,
    AvailabilityZone=subnet_configs[1]['az']
)
ec2_client.create_tags(Resources=[public_subnet2.id],Tags=[{'Key': 'Name', 'Value': subnet_configs[1]['tag']}])

In [ ]:
private_subnet1 = ec2_resource.create_subnet(
    CidrBlock=subnet_configs[2]['cidr_block'],
    VpcId=vpc.id,
    AvailabilityZone=subnet_configs[2]['az']
)
ec2_client.create_tags(Resources=[private_subnet1.id],Tags=[{'Key': 'Name', 'Value': subnet_configs[2]['tag']}])

private_subnet2 = ec2_resource.create_subnet(
    CidrBlock=subnet_configs[3]['cidr_block'],
    VpcId=vpc.id,
    AvailabilityZone=subnet_configs[3]['az']
)
ec2_client.create_tags(Resources=[private_subnet2.id],Tags=[{'Key': 'Name', 'Value': subnet_configs[3]['tag']}])

In [ ]:
sg_name = 'htts-sg'
# ec2.create_security_group(sg_name, vpc_id)

In [ ]:
# response = ec2_client.describe_security_group_rules(SecurityGroupRuleIds=['sgr-0151050acf52e2f6b'])

In [ ]:
ec2_instance_type = os.environ['AWS_DEFAULT_INSTANCE_TYPE'] # t2.micro
key_name='AMominNJ'
sg_id = 'sg-03866c6a844806569'
ami_id = 'ami-0e86e20dae9224db8' # Ubuntu Server 24.04 LTS (Free Tier Eligible)

ec2_in_public_subnet = ec2_client.run_instances(
    ImageId=ami_id,                         # The ID of the AMI
    InstanceType=ec2_instance_type,         # The instance type
    KeyName=key_name,                       # The name of the key pair
    MaxCount=1,                             # Number of instances to launch
    MinCount=1,
    NetworkInterfaces=[{
        'SubnetId': public_subnet1.subnet_id,   # Subnet within the VPC
        'DeviceIndex': 0,                       # Primary network interface
        'AssociatePublicIpAddress': True,       # Automatically assign public IP
        'Groups': [sg_id]                       # Security group ID(s)
    }]
)

In [ ]:
# Create Internet Gateway and attach that with VPC
igw_id = ec2_resource.create_internet_gateway()
ec2_client.attach_internet_gateway(InternetGatewayId=igw_id,VpcId=vpc_id)

In [ ]:
public_route_table = ec2_resource.create_route_table(VpcId=vpc_id)
ec2_client.create_tags(Resources=[public_route_table.id],Tags=[{'Key': 'Name','Value': 'public_route_table'},])

public_route_table.associate_with_subnet(SubnetId=public_subnet1.id)

route_params = {'DestinationCidrBlock': '0.0.0.0/0', 'GatewayId': igw_id}
public_route_table.create_route(**route_params)

In [ ]:
private_route_table = ec2_resource.create_route_table(VpcId=vpc_id)
ec2_client.create_tags(Resources=[private_route_table.id],Tags=[{'Key': 'Name','Value': 'private_route_table'},])

private_route_table.associate_with_subnet(SubnetId=private_subnet1.id)

- YOU SHOULD BE ABLE TO CONNECT TO YOUR EC2 INSTANCE (`ec2_in_public_subnet`).

In [ ]:
ec2_instance_type = os.environ['AWS_DEFAULT_INSTANCE_TYPE'] # t2.micro
key_name='AMominNJ'
sg_id = 'sg-03866c6a844806569'
ami_id = 'ami-0e86e20dae9224db8' # Ubuntu Server 24.04 LTS (Free Tier Eligible)

ec2_in_private_subnet = ec2_client.run_instances(
    ImageId=ami_id,                # The ID of the AMI
    InstanceType=ec2_instance_type,       # The instance type
    KeyName=key_name,              # The name of the key pair
    MaxCount=1,                    # Number of instances to launch
    MinCount=1,
    NetworkInterfaces=[{
        'SubnetId': private_subnet1.subnet_id,     # Subnet within the VPC
        'DeviceIndex': 0,          # Primary network interface
        'AssociatePublicIpAddress': True,  # Automatically assign public IP
        'Groups': [sg_id]  # Security group ID(s)
    }]
)

In [ ]:
ec2_client.create_tags(Resources=['i-0169495b08d0ec48c'],Tags=[{'Key': 'Name','Value': 'ec2_in_private_subnet'},])

- YOU SHOULD BE ABLE TO CONNECT TO YOUR `ec2_in_private_subnet` EC2 INSTANCE from `ec2_in_public_subnet` EC2 INSTANCE. SINCE ROUTE TABLE ALLOW INTER SUBNET COMMUNICATION.
- BUT YOU SHOULD NOT BE ABLE TO CONNECT TO YOUR `ec2_in_private_subnet` EC2 INSTANCE from `LOCAL_MACHINE` EC2 INSTANCE.
- BUT HOW DOES `ec2_in_private_subnet` UPDATES IT'S PACAGES SINCE IT DOES NOT HAVE INTERNET ACCESS? `NAT Gateway`!!!

In [ ]:
# Step 1: Allocate an Elastic IP address
eip_response = ec2_client.allocate_address(Domain='vpc')
eip_allocation_id = eip_response['AllocationId']
print(f"Elastic IP allocated: {eip_allocation_id}")

# Step 2: Create the NAT Gateway
nat_gateway_response = ec2_client.create_nat_gateway(
    SubnetId=public_subnet1.id,
    AllocationId=eip_allocation_id
)
nat_gateway_id = nat_gateway_response['NatGateway']['NatGatewayId']
print(f"NAT Gateway created: {nat_gateway_id}")

In [ ]:
private_route_params = {'DestinationCidrBlock': '0.0.0.0/0', 'GatewayId': nat_gateway_id}
private_route_table.create_route(**private_route_params)

In [ ]:
public_ip = ''  # Collect it from instetiated EC2 instance
private_ip = '' # Collect it from instetiated EC2 instance

```sh
# Run to access into ec2_in_public_subnet from local machine
ssh -o StrictHostKeyChecking=no -i ~/.ssh/{key_name}.pem ubuntu@{public_ip}
# Run to copy key file into ec2_in_public_subnet from local machine
scp -i ~/.ssh/{key_name}.pem ~/.ssh/{key_name}.pem ubuntu@{public_ip}:.ssh/
# Run to access into ec2_in_private_subnet from ec2_in_public_subnet
ssh -o StrictHostKeyChecking=no -i ~/.ssh/{key_name}.pem ubuntu@{private_ip}
```

In [ ]:
# vpc.delete_vpc_with_dependencies(vpc_id)

#### [BeBetterDev: AWS VPC Endpoints](https://www.youtube.com/watch?v=jo3X_aay4Vs&list=PL9nWRykSBSFhzwFa9Z5PqBM0XON8hpXE7&index=6)

-   `Gateway` endpoints serve as a target for a route in your route table for traffic destined for the service.

In [ ]:
# VPC Endpoint parameters
vpc_endpoint_tag = 'rs-glue-vpc-endpoint'
service_name = 'com.amazonaws.us-east-1.s3'  # Replace with the desired service (e.g., S3)
subnet_ids = ['subnet-0980ad10eb313405b', 'subnet-0de97821ddb8236f7', 'subnet-0a160fbe0fcafe373', 'subnet-0ca765b361e4cb186', 'subnet-0a972b05a5b162feb']  # List of subnets where the endpoint should be placed
security_group_ids = [sg_id]  # Security group(s) associated with the endpoint
route_table_ids = ['rtb-0ec4311296ec952f8']

# Create an Interface Endpoint
vpc_endpoint = ec2_client.create_vpc_endpoint(
    VpcEndpointType='Gateway',
    VpcId=vpc_id,
    ServiceName=service_name,
    RouteTableIds=route_table_ids,
    # SubnetIds=sg_id,
    # SecurityGroupIds=security_group_ids,
    PrivateDnsEnabled=False  # Enable private DNS to resolve service names within the VPC
)
print(vpc_endpoint)

In [ ]:
# ec2_client.create_tags(Resources=['vpce-039138a9e51a77069'],Tags=[{'Key': 'Name', 'Value': vpc_endpoint_tag}])

In [ ]:
vpc_endpoint_id = vpc_endpoint['VpcEndpoint']['VpcEndpointId']
# Delete the VPC Endpoint
response = ec2_client.delete_vpc_endpoints(VpcEndpointIds=[vpc_endpoint_id])

#### Experiments

- Create a Private VPC
- Create a Internet Gatway and attach it to the VPC

In [ ]:
# ?ec2_resource.create_vpc

In [ ]:
tag_specifications=[
    {
        'Tags': [
            {
                'Key': 'Name',
                'Value': 'httx-vpc'
            },
        ]
    },
]
vpc_cidr_block = '172.0.0.0/16'
vpc = ec2_resource.create_vpc(CidrBlock=vpc_cidr_block, TagSpecifications=tag_specifications)
vpc.wait_until_available()
print(f"VPC '{vpc.id}' created with CIDR block '{vpc_cidr_block}'")


# Create Internet Gateway and attach that with VPC
igw = ec2_resource.create_internet_gateway()

# attach the IG with VPC
vpc.attach_internet_gateway(InternetGatewayId=igw.id)
print(f"Internet Gateway '{igw.id}' attached to VPC '{vpc.id}'")


- Create Subnets and Route Tables within the VPC.
- Associate Route Tables with the Subnets.

In [ ]:
subnet_configs = [
    {'cidr_block': '172.0.1.0/24', 'az': 'us-east-1a'},
    {'cidr_block': '172.0.2.0/24', 'az': 'us-east-1b'}
]

route_table_configs = [
    {
        'routes': [
            {'destination_cidr_block': '0.0.0.0/0', 'gateway_id': igw.id, 'nat_gateway_id': ''}
        ]
    }
]


for subnet_config in subnet_configs:

    subnet = ec2_resource.create_subnet(
        CidrBlock=subnet_config['cidr_block'],
        VpcId=vpc.id,
        AvailabilityZone=subnet_config['az']
    )

    route_table = ec2_resource.create_route_table(VpcId=vpc.id)
    route_table_config = route_table_configs[0]

    if 'routes' in route_table_config:
        for route in route_table_config['routes']:
            route_params = {'DestinationCidrBlock': route['destination_cidr_block']}
            
            if 'gateway_id' in route and route['gateway_id']: route_params['GatewayId'] = route['gateway_id']
            elif igw.id: route_params['GatewayId'] = igw.id
            
            if 'nat_gateway_id' in route and route['nat_gateway_id']:
                route_params['NatGatewayId'] = route['nat_gateway_id']
            
            # Create a route within the Route Table
            route_table.create_route(**route_params)
    
    route_table.associate_with_subnet(SubnetId=subnet.id)
    print(f"Route table '{route_table.id}' associated with subnet '{subnet.id}'")

In [ ]:
security_group_configs = [
    {
        'group_name': f"httx-{date.today().strftime('%Y-%m-%d')}-sg",
        'description': f"httx-{date.today().strftime('%Y-%m-%d')} security group",
        'ingress_rules': [
            {'protocol': 'tcp', 'cidr_ip': '0.0.0.0/0', 'from_port': 80, 'to_port': 80},
            {'protocol': 'tcp', 'cidr_ip': '0.0.0.0/0', 'from_port': 443, 'to_port': 443}
        ],
        'egress_rules': [
            {'protocol': 'tcp', 'cidr_ip': '0.0.0.0/0', 'from_port': 0, 'to_port': 0}
        ]
    }
]

if security_group_configs:
    for sg_config in security_group_configs:
        security_group = ec2_resource.create_security_group(
            GroupName=sg_config['group_name'],
            Description=sg_config['description'],
            VpcId=vpc.id
        )
        if 'ingress_rules' in sg_config:
            for rule in sg_config['ingress_rules']:
                security_group.authorize_ingress(
                    IpProtocol=rule['protocol'],
                    CidrIp=rule['cidr_ip'],
                    FromPort=rule['from_port'],
                    ToPort=rule['to_port']
                )
        if 'egress_rules' in sg_config:
            for rule in sg_config['egress_rules']:
                security_group.authorize_egress(
                    IpPermissions=[{
                        'IpProtocol': rule['protocol'],
                        'IpRanges': [{'CidrIp': rule['cidr_ip']}],
                        'FromPort': rule['from_port'],
                        'ToPort': rule['to_port']
                    }]
                )
        print(f"Security group '{security_group.id}' created with name '{sg_config['group_name']}'")


-   Delete all resources associated with the VPC.
-   Then delete the VPC.

In [ ]:
vpc = ec2_resource.Vpc(vpc.id)

# Detach and delete all internet gateways
for igw in vpc.internet_gateways.all():
    vpc.detach_internet_gateway(InternetGatewayId=igw.id)
    # Delete the Internet Gateway
    igw.delete()
    # ec2_client.delete_internet_gateway(InternetGatewayId=igw.id)
    print(f"Deleted Internet Gateway: {igw.id}")

# Delete all route table associations and route tables
for rt in vpc.route_tables.all():
    for association in rt.associations:
        if not association.main:
            ec2_client.disassociate_route_table(AssociationId=association.id)
    if not rt.associations_attribute or all(not assoc['Main'] for assoc in rt.associations_attribute):
        rt.delete()
        print(f"Deleted Route Table: {rt.id}")

# Delete all network ACLs (except the default one)
for acl in vpc.network_acls.all():
    if not acl.is_default:
        acl.delete()
        print(f"Deleted Network ACL: {acl.id}")

# Delete all subnets
for subnet in vpc.subnets.all():
    for eni in subnet.network_interfaces.all():
        eni.delete()
        print(f"Deleted Network Interface: {eni.id}")
    subnet.delete()
    print(f"Deleted Subnet: {subnet.id}")

# Delete all security groups (except the default one)
for sg in vpc.security_groups.all():
    if sg.group_name != 'default':
        sg.delete()
        print(f"Deleted Security Group: {sg.id}")

# Finally, delete the VPC
vpc.delete()
print(f"Deleted VPC: {vpc.id}")


In [ ]:
# ?ec2_client.describe_vpcs

In [ ]:
response = ec2_client.describe_vpcs(
    VpcIds=[
        'vpc-054e315c7fbdb0d6f',
    ]
)
print(response)